# Import & Load CASEN banking data

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

In [2]:
remote_path = 'http://pacha.datawheel.us/economia/casen/analysis/5_banking/'
local_path = '../data/'

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, inline_dimension_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load data

In [4]:
df1 = download_file(remote_path,local_path,'banking_products.csv')
df1 = df1.astype({'debit_card_id':'int','credit_card_bank_id':'int','credit_card_store_id':'int','checks_id':'int','credit_line_id':'int','year':'int','comuna_datachile_id':'int'})
list(df1)

Already downloaded. Using: ../data/banking_products.csv
Encoding: ascii


['debit_card_id',
 'credit_card_bank_id',
 'credit_card_store_id',
 'checks_id',
 'credit_line_id',
 'expc',
 'expr',
 'year',
 'comuna_datachile_id']

In [5]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df1, 'environment', 'fact_banking_casen')

DROP TABLE IF EXISTS environment.fact_banking_casen;
CREATE TABLE "environment"."fact_banking_casen" (
"debit_card_id" INTEGER,
  "credit_card_bank_id" INTEGER,
  "credit_card_store_id" INTEGER,
  "checks_id" INTEGER,
  "credit_line_id" INTEGER,
  "expc" INTEGER,
  "expr" INTEGER,
  "year" INTEGER,
  "comuna_datachile_id" INTEGER
)
COPY "environment"."fact_banking_casen" ("debit_card_id","credit_card_bank_id","credit_card_store_id","checks_id","credit_line_id","expc","expr","year","comuna_datachile_id") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Updates

In [6]:
engine.execute("""
ALTER TABLE environment.fact_banking_casen
  ADD COLUMN date_id INTEGER; 
""")

engine.execute("""
UPDATE environment.fact_banking_casen
SET date_id = dim_date.id
FROM public.dim_date
WHERE dim_date.the_year = environment.fact_banking_casen.year
      AND dim_date.month_of_year = 1
      AND dim_date.day_of_month = 1
""")


### Dims

In [7]:
d2 = download_file(remote_path,local_path,'banking_products_id.csv')
print (inline_table_xml(d2, 'reply', 'reply_id', 'reply'))

Already downloaded. Using: ../data/banking_products_id.csv
Encoding: ascii

<InlineTable alias="reply">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Si</Value>
      <Value column="es_description">Si</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">No</Value>
      <Value column="es_description">No</Value>
    </Row>
  <Row>
      <Value column="id">0</Value>
      <Value column="description">nan</Value>
      <Value column="es_description">nan</Value>
    </Row>
  </Rows>
</InlineTable>
    
